<a href="https://colab.research.google.com/github/Gabriellrs/movieRecommender/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommender

Neste projeto, decidi criar um sistema de recomendação de filmes inspirado em plataformas como a Netflix. Para isso, utilizo o dataset MovieLens, que contém informações sobre filmes, como títulos e gêneros. Esta célula é responsável por baixar e extrair o dataset automaticamente, garantindo que todas as informações necessárias estejam disponíveis. Após a execução, os arquivos extraídos são listados.

In [ ]:
import requests
import zipfile
import os

# Função para baixar e extrair o dataset MovieLens
def download_and_extract_dataset():
    url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
    output_zip = "ml-latest-small.zip"

    print("Baixando o dataset...")
    response = requests.get(url)
    with open(output_zip, "wb") as f:
        f.write(response.content)
    print("Download concluído.")

    print("Extraindo os arquivos...")
    with zipfile.ZipFile(output_zip, "r") as zip_ref:
        zip_ref.extractall("ml-latest-small")
    print("Extração concluída.")

    # Listar os arquivos extraídos
    extracted_files = os.listdir("ml-latest-small")
    print(f"Arquivos extraídos: {extracted_files}")

    # Retornar o diretório completo
    return os.path.join("ml-latest-small", extracted_files[0])  # Subdiretório criado pelo ZIP

# Baixar e carregar os arquivos do dataset
dataset_path = download_and_extract_dataset()


Baixando o dataset...
Download concluído.
Extraindo os arquivos...
Extração concluída.
Arquivos extraídos: ['ml-latest-small']


Esse é core do sistema de recomendação. Ele utiliza uma abordagem baseada no conteúdo (Content-Based Filtering), que considera informações como os títulos e gêneros dos filmes para sugerir opções semelhantes ao que o usuário escolhe. Nesta célula, carrego o dataset, preparo os dados, calculo a similaridade entre os filmes e implemento uma função de recomendação. No final, é possível testar o sistema digitando o título de um filme para receber sugestões personalizadas.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Caminho para o arquivo movies.csv
movies_file = os.path.join(dataset_path, "movies.csv")

# Verificar se o arquivo existe
if not os.path.exists(movies_file):
    raise FileNotFoundError(f"O arquivo 'movies.csv' não foi encontrado no diretório '{dataset_path}'.")

# 1. Carregar os dados do arquivo movies.csv
movies = pd.read_csv(movies_file)

# 2. Pré-processamento dos dados
movies['genres'] = movies['genres'].fillna('')
movies['combined_features'] = movies['title'] + " " + movies['genres']

# 3. Vetorização TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(movies['combined_features'])

# 4. Calcular a similaridade do cosseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 5. Função de recomendação
def recommend_movies(title, cosine_sim=cosine_sim):
    indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
    idx = indices.get(title, None)

    if idx is None:
        return ["Filme não encontrado. Tente outro título."]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]

    return movies['title'].iloc[movie_indices].tolist()

# 6. Interface para testar o sistema
print("Digite o título de um filme (exemplo: 'Toy Story (1995)'):")
user_input = input()

recommendations = recommend_movies(user_input)

print(f"\nFilmes recomendados para '{user_input}':")
for movie in recommendations:
    print(movie)


Digite o título de um filme (exemplo: 'Toy Story (1995)'):
Baywatch (2017)

Filmes recomendados para 'Baywatch (2017)':
Get Out (2017)
It (2017)
Call Me by Your Name (2017)
The House (2017)
CHiPS (2017)
Snatched (2017)
Free Fire (2017)
Gintama (2017)
Geostorm (2017)
Win It All (2017)


Se estiver tendo problemas pra achar algum filme que esteja no dataset, digite a primeira letra do filme e essa função vai listar todos presentes no dataset com essa inicial.

In [ ]:
def listar_filmes_inicial(letra, movies=movies):
    # Garantir que a letra esteja em maiúscula para a comparação
    letra = letra.upper()

    # Filtrar filmes que começam com a letra fornecida
    filmes_filtrados = movies[movies['title'].str.startswith(letra, na=False)]

    if filmes_filtrados.empty:
        return f"Não há filmes no dataset que comecem com a letra '{letra}'."

    # Retornar a lista de filmes
    return filmes_filtrados['title'].tolist()

# Exemplo de uso:
# Solicitar ao usuário a inicial do filme
print("Digite a inicial de um filme (exemplo: 'T' para filmes que começam com T):")
inicial = input().strip()

# Listar filmes com a inicial fornecida
resultados = listar_filmes_inicial(inicial)

# Exibir os resultados
print(f"\nFilmes que começam com '{inicial}':")
for filme in resultados:
    print(filme)


Digite a inicial de um filme (exemplo: 'T' para filmes que começam com T):
t

Filmes que começam com 't':
Toy Story (1995)
Tom and Huck (1995)
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
To Die For (1995)
Two if by Sea (1996)
Things to Do in Denver When You're Dead (1995)
Taxi Driver (1976)
Three Wishes (1995)
Total Eclipse (1995)
To Wong Foo, Thanks for Everything! Julie Newmar (1995)
Three Colors: Red (Trois couleurs: Rouge) (1994)
Three Colors: Blue (Trois couleurs: Bleu) (1993)
Three Colors: White (Trzy kolory: Bialy) (1994)
To Live (Huozhe) (1994)
Tank Girl (1995)
Tales from the Crypt Presents: Demon Knight (1995)
Tales from the Hood (1995)
Tom & Viv (1994)
Tommy Boy (1995)
Timecop (1994)
True Lies (1994)
Terminal Velocity (1994)
Thirty-Two Short Films About Glenn Gould (1993)
Threesome (1994)
Three Musketeers, The (1993)
Tombstone (1993)
True Romance (1993)
Terminator 2: Judgment Day (1991)
Two Much (1995)
Thin Line Between Love and Hate, A (1996)
Theodore Rex (1995)
True Crime (19